In [ ]:
import pandas as pd
import csv
from loader import *
from pair_finder import *
import time
from random import seed
import keyboard
from random import seed
from random import randint

In [ ]:
ratings_path_s = "DATA\\ratings.csv"
ratings_path = "DATA\\ratings.csv"
movies_path =  "DATA\\movies.csv"

MinScore = 4
MaxCombo = 4
MinFrequency = 0.1
ratings_stream = createRatingsStream(ratings_path_s, MinScore)
userBaskets, movie_list = CreateMovieBaskets(ratings_path, MinScore, 1000)
movies_df = ReadMovies(movies_path, movie_list)

In [ ]:
myApriori = myApriori(userBaskets, MinFrequency, MaxCombo)
c_combos = export_combos(myApriori)
s_combos = sampledApriori(100, ratings_stream, MinFrequency, MaxCombo)
print(get_scores(c_combos, s_combos))

In [10]:
x = {}
x[4] = {"A", "B", "C", "D"}
x[3] = [{"A","B","C"}, {"A","C","D"}, {"A","B","D"}, {"B","C","D"}]
x[2] = [{"A","B"}, {"A","C"}, {"A","D"}, {"B", "C"}, {"B","D"}, {"C","D"}]
x[1] = [{"A"}, {"B"}, {"C"}, {"D"}]

In [ ]:
len(x[4])/2

In [13]:
for k in range(2, len(x) + 1): # PICK K-SET from itemsets
    
    for item in x[k]: # item from itemset
        item_rules = []
        for splitting in range(1, round(k / 2)): # Calculate combinations to the middle 
                                                 # the other are symmetric
            



2
3
4


In [78]:
# PRODUCES THE HALF SIZE-COMBINATIONS
# FOR THE CURRENT SET
# THE OTHER HALF ARE SYMMETRIC

import math
y = frozenset(["A", "B", "C", "D", "E"])

y_max = len(y)
combo_size = len(y)
even_size = (combo_size % 2) == 0
mid = math.ceil(combo_size / 2)

if even_size:
    mid += 1
s_y = sorted(list(y))

rules = {}
for i in range(1, mid):
    rules[i] = []
    final_hypothesis =  list(range(combo_size - i,combo_size))
    current_hypothesis = list(range(i))
    
    #print(final_hypothesis)
    #print(current_hypothesis)
    while True:
        print(current_hypothesis)
        
        hypothesis = frozenset([s_y[i] for i in current_hypothesis])
        conclusion = y.difference(hypothesis)
        tmp_rule = {
            'hypothesis': hypothesis,
            'conclusion': conclusion    
        }
        rules[i].append(tmp_rule)
        print(hypothesis)
        if next_hypothesis(current_hypothesis, final_hypothesis) == -1:
            break

[0]
frozenset({'A'})
[1]
frozenset({'B'})
[2]
frozenset({'C'})
[3]
frozenset({'D'})
[4]
frozenset({'E'})
Already at final hypothesis!!
[0, 1]
frozenset({'B', 'A'})
[0, 2]
frozenset({'C', 'A'})
[0, 3]
frozenset({'D', 'A'})
[0, 4]
frozenset({'E', 'A'})
[1, 2]
frozenset({'C', 'B'})
[1, 3]
frozenset({'D', 'B'})
[1, 4]
frozenset({'B', 'E'})
[2, 3]
frozenset({'C', 'D'})
[2, 4]
frozenset({'C', 'E'})
[3, 4]
frozenset({'D', 'E'})
Already at final hypothesis!!


In [79]:
rules

{1: [{'hypothesis': frozenset({'A'}),
   'conclusion': frozenset({'B', 'C', 'D', 'E'})},
  {'hypothesis': frozenset({'B'}),
   'conclusion': frozenset({'A', 'C', 'D', 'E'})},
  {'hypothesis': frozenset({'C'}),
   'conclusion': frozenset({'A', 'B', 'D', 'E'})},
  {'hypothesis': frozenset({'D'}),
   'conclusion': frozenset({'A', 'B', 'C', 'E'})},
  {'hypothesis': frozenset({'E'}),
   'conclusion': frozenset({'A', 'B', 'C', 'D'})}],
 2: [{'hypothesis': frozenset({'A', 'B'}),
   'conclusion': frozenset({'C', 'D', 'E'})},
  {'hypothesis': frozenset({'A', 'C'}),
   'conclusion': frozenset({'B', 'D', 'E'})},
  {'hypothesis': frozenset({'A', 'D'}),
   'conclusion': frozenset({'B', 'C', 'E'})},
  {'hypothesis': frozenset({'A', 'E'}),
   'conclusion': frozenset({'B', 'C', 'D'})},
  {'hypothesis': frozenset({'B', 'C'}),
   'conclusion': frozenset({'A', 'D', 'E'})},
  {'hypothesis': frozenset({'B', 'D'}),
   'conclusion': frozenset({'A', 'C', 'E'})},
  {'hypothesis': frozenset({'B', 'E'}),
   'con

In [65]:




def next_hypothesis(current, final):
    h_size = len(current)
    pointer = h_size - 1
    if current == final:
        print("Already at final hypothesis!!")
        return -1
    #print(h_size)
    while True:
        if current[pointer] != final[pointer]:
        
            current[pointer] += 1
            for i in range(pointer + 1, h_size):
                current[i] = current[i-1] + 1
            break
        else:
            pointer -= 1

In [68]:
current = [0,1,2]
final = [2,3,4]

count = 0
while next_hypothesis(current, final) != -1:
    print(current)
    count += 1
    print(count)

[0, 1, 3]
1
[0, 1, 4]
2
[0, 2, 3]
3
[0, 2, 4]
4
[0, 3, 4]
5
[1, 2, 3]
6
[1, 2, 4]
7
[1, 3, 4]
8
[2, 3, 4]
9
Already at final hypothesis!!


In [46]:
for i in range(9,-1, -1):
    print(i)

9
8
7
6
5
4
3
2
1
0


STORING DATA EXAMPLE

In [8]:
import pandas as pd
dict_2 = {
    'itemset': '(a,b)',
    'rule': 'a-->b',
    'hypothesis': 'a',
    'conclusion': 'b',
    'confidence': 0.1,
    'lift': 1.0,
    'interest': 0.2,
    'rule_ID': 2
}
df2 = pd.DataFrame(dict_2, index=[0])
df2